In [1]:
import urllib.request
import json
import numpy as np
import pandas as pd
from joblib import load
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [2]:
def custom_analyzer(doc):
    return doc

cardVec = load('matrix_data/cardVec_2k_20dim.joblib')
deckVecorizer = load('matrix_data/deckVectorizer_2k_8K.joblib')

In [3]:
deckVecorizer

TfidfVectorizer(analyzer=<function custom_analyzer at 0x0000024F7FE65AF0>,
                lowercase=False,
                vocabulary=['Strike_R', 'Bash', 'Anger', 'Body Slam', 'Clash',
                            'Cleave', 'Clothesline', 'Headbutt', 'Heavy Blade',
                            'Iron Wave', 'Perfected Strike', 'Pommel Strike',
                            'Sword Boomerang', 'Thunderclap', 'Twin Strike',
                            'Wild Strike', 'Blood for Blood', 'Carnage',
                            'Dropkick', 'Hemokinesis', 'Pummel', 'Rampage',
                            'Reckless Charge', 'Searing Blow', 'Sever Soul',
                            'Uppercut', 'Whirlwind', 'Bludgeon', 'Feed',
                            'Fiend Fire', ...])

In [4]:
data_url = 'https://raw.githubusercontent.com/KU81-ComSci-ProjectEnd-GameCard/main/master/Predictor/data/processed_data/picked_2k_8k.json'
with urllib.request.urlopen(data_url) as url:
    pick_js = json.loads(url.read().decode())
pick = pd.DataFrame(pick_js)

In [5]:
pick

,deck,picked
0,"[Strike_R, Strike_R, Strike_R, Strike_R, Strik...",Bludgeon
1,"[Strike_R, Strike_R, Strike_R, Strike_R, Strik...",Flex
2,"[Strike_R, Strike_R, Strike_R, Strike_R, Strik...",Anger
3,"[Strike_R, Strike_R, Strike_R, Strike_R, Strik...",Reckless Charge
4,"[Strike_R, Strike_R, Strike_R, Strike_R, Strik...",Flex
...,...,...
7986,"[Strike_R, Strike_R, Strike_R, Strike_R, Strik...",Power Through
7987,"[Strike_R, Strike_R, Strike_R, Strike_R, Strik...",Barricade
7988,"[Strike_R, Strike_R, Strike_R, Strike_R, Strik...",Shrug It Off+1
7989,"[Strike_R, Strike_R, Strike_R, Strike_R, Strik...",Infernal Blade


troch

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [21]:
X = torch.tensor(deck_matrix, dtype=torch.float32)
pickToMatrix = list()
for card in pick.picked:
    pickToMatrix.append(card_matrix[card])
y = torch.tensor(pickToMatrix, dtype=torch.float32)

NameError: name 'deck_matrix' is not defined

In [ ]:
# Create DataLoader for training
train_dataset = TensorDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(257, 128)  # Input layer to hidden layer
        self.fc2 = nn.Linear(128, 64)   # Hidden layer to another hidden layer
        self.fc3 = nn.Linear(64, 40)    # Hidden layer to output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Instantiate the model, define the loss function and the optimizer
model = SimpleNN()
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for X_batch, Y_batch in train_loader:
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss for every epoch
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Evaluate the model (optional step)
with torch.no_grad():
    model.eval()
    test_outputs = model(X)
    test_loss = criterion(test_outputs, y)
    print(f'Test Loss: {test_loss.item():.4f}')

kerass

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense, Input

In [7]:
X = deckVecorizer.fit_transform(pick.deck.tolist())
X = X.toarray()
y = [cardVec[picked] for picked in pick.picked ]
y_card = pick.picked 

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)
y_card = np.array(y_card, dtype= str)

# Split data into training and testing sets
X_train, X_test, y_train, y_test, yC_train,yC_test = train_test_split(X, y,y_card, test_size=0.2, random_state=42)

In [8]:
X.shape

(7991, 257)

In [9]:
y.shape

(7991, 20)

In [10]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [42]:
model = Sequential([
    Input(shape=(257,)),               # Define the input shape
    Dense(128, activation='relu'),     # First hidden layer
    Dense(64, activation='relu'),      # Second hidden layer
    Dense(20)           
])
# Compile the model
model.compile(optimizer='adam', 
              loss='mse', 
              metrics=['mae']) 

In [43]:
class PrintLossEveryTenEpochs(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch + 1}, Loss: {logs["loss"]:.4f}')

In [44]:
# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=100, verbose=0, callbacks=[PrintLossEveryTenEpochs()])

Epoch 100, Loss: 0.0765
Epoch 200, Loss: 0.0668
Epoch 300, Loss: 0.0632
Epoch 400, Loss: 0.0611
Epoch 500, Loss: 0.0597


In [26]:
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    
    similarity = dot_product / (norm1 * norm2)
    return similarity

In [45]:

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test, verbose=1)

# If loss is a list, print the first element as the test loss
print(f'Test Loss: {loss[0]:.4f}' if isinstance(loss, list) else f'Test Loss: {loss:.4f}')

# Make predictions (optional)
predictions = model.predict(X_test)
simility_test = list()
for pred, pick_card in zip(predictions , yC_test):
    simility_test.append(cosine_similarity(pred, cardVec[pick_card]))

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - loss: 0.2207 - mae: 0.3197
Test Loss: 0.2276
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [46]:
mt = pd.DataFrame()

In [47]:
mt['similate'] = simility_test

In [48]:
mt

,similate
0,0.745252
1,0.675095
2,0.809485
3,0.492102
4,0.918513
...,...
1594,0.741505
1595,0.472340
1596,0.810608
1597,0.600932


In [53]:
mt.loc[ mt.similate > 0.7].count()

similate    694
dtype: int64